In [1]:
import gzip
import json
import os
import io
import csv #to read in and split up all the content from the dataset input file
import numpy as np #import numpy using np as an alias
import pandas as pd #import pandas using pd as an alias
import matplotlib.pyplot as plt #import matplotlib.pyplot using plt as an alias
import seaborn as sb #import seaborn using sns as an alias
import sys
import codecs
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #import the encoder classes

In [2]:
draftDf = pd.read_csv('raw_training_data.csv', delimiter=',')

In [22]:
def jsonStrToList(jsonStr):
    jsonListOfDict = json.loads(jsonStr)
    return jsonListOfDict

In [23]:
#Store current hp for floor 33 in hpFloor33 and store all indices for entries with less than 33 floors
#in deleteIndices to remove from dataset.
currentHpFloor33 = []
deleteIndices = []
for floorsStr in draftDf['current_hp_per_floor']:
    hpPerFloorListOfInts = jsonStrToList(floorsStr)
    try:
        currentHpFloor33.append(hpPerFloorListOfInts[32])
    except:
        deleteIndices.append(len(deleteIndices) + len(currentHpFloor33))

In [24]:
maxHpFloor33 = []
deleteIndices1 = []
for floorsStr in draftDf['current_hp_per_floor']:
    hpPerFloorListOfInts = jsonStrToList(floorsStr)
    try:
        maxHpFloor33.append(hpPerFloorListOfInts[32])
    except:
        deleteIndices1.append(len(deleteIndices1) + len(maxHpFloor33))

In [25]:
#convert victory columns from TRUE/FALSE to binary
victories = []
deleteIndices2 = []
countRow = 0
for victory in draftDf['victory']:
    if (victory == True):
        victories.append(1)
    elif (victory == False):
        victories.append(0)
    else:
        deleteIndices2.append(countRow)
    countRow += 1

In [26]:
#convert ascension_level column from str to int and store the ints in ascensionLvl
ascensionLvl = []
deleteIndices3 = []
countRow = 0
for ascension in draftDf['ascension_level']:
    if (ascension < 0):
        deleteIndices3.append(countRow)
    else:
        ascensionLvl.append(int(ascension))
        
len(deleteIndices3)

4

In [27]:
uniqueCards = ['Defend', 'Neutralize', 'Strike', 'Survivor', 'Acrobatics', 'Backflip', 'Bane', 
               'Blade Dance', 'Cloak And Dagger', 'Dagger Spray', 'Dagger Throw', 'Deadly Poison', 
               'Deflect', 'Dodge and Roll', 'Flying Knee', 'Outmaneuver', 'PiercingWail', 
               'Poisoned Stab', 'Prepared', 'Quick Slash', 'Slice', 'Sneaky Strike', 'Sucker Punch', 
               'Accuracy', 'All Out Attack', 'Backstab', 'Blur', 'Bouncing Flask', 
               'Calculated Gamble', 'Caltrops', 'Catalyst', 'Choke', 'Concentrate', 
               'Crippling Cloud', 'Dash', 'Distraction', 'Endless Agony', 'Escape Plan', 
               'Eviscerate', 'Expertise', 'Finisher', 'Flechettes', 'Footwork', 'Heel Hook', 
               'Infinite Blades', 'Leg Sweep', 'Masterful Stab', 'Noxious Fumes', 'Predator', 
               'Reflex', 'Riddle With Holes', 'Setup', 'Skewer', 'Tactician', 'Terror', 
               'Well Laid Plans', 'A Thousand Cuts', 'Adrenaline', 'After Image', 'Alchemize', 
               'Bullet Time', 'Burst', 'Corpse Explosion', 'Die Die Die', 'Doppelganger', 'Envenom', 
               'Glass Knife', 'Grand Finale', 'Malaise', 'Nightmare', 'Phantasmal Killer', 
               'Storm of Steel', 'Tools of the Trade', 'Unload', 'Wraith Form', 'Bandage Up',
               'Blind', 'Dark Shackles', 'Deep Breath', 'Discovery', 'Dramatic Entrance',
               'Enlightenment', 'Finesse', 'Flash of Steel', 'Forethought', 'Good Instincts',
               'Impatience', 'Jack Of All Trades', 'Madness', 'Mind Blast', 'Panacea',
               'PanicButton', 'Purity', 'Swift Strike', 'Trip', 'Apotheosis', 'Chrysalis',
               'HandOfGreed', 'Magnetism', 'Master of Strategy', 'Mayhem', 'Metamorphosis',
               'Panache', 'Sadistic Nature', 'Secret Technique', 'Secret Weapon', 'The Bomb',
               'Thinking Ahead', 'Transmutation', 'Violence', 'Apparition', 'Beta', 'Bite',
               'Expunger', 'Insight', 'J.A.X.', 'Miracle', 'Omega', 'RitualDagger', 'Safety',
               'Shiv', 'Smite', 'Through Violence', 'Clumsy', 'Decay', 'Doubt',
               'Injury', 'Normality', 'Pain', 'Parasite', 'Regret',
               'Shame', 'Writhe', 'Pride', 'Venomology', 'Underhanded Strike', 'Crippling Poison',
               'Night Terror']
#store all indices for rows with cards that do not belong to silent class to later delete these rows
deleteIndices4 = []
index = 0
indexDelete = False
for deck in draftDf['master_deck']:
    cardList = deck.strip("][").split(', ')
    cardList1 = []
    #clean up each card
    for card in cardList:
        card = card.strip('""')
        cardList1.append(card.strip("''"))
    #store indices for the rows with a non-silent card in their deck
    for card in cardList1:
        cardSplit = []
        #if card is not in uniqueCards
        if not(card in uniqueCards):
            #some cards are inconsistently labeled,
            #if underscores are found then the card can
            #be regular or upgraded
            if ("_" in card):
                cardSplit = card.split("_")
                #if card is upgraded
                if (cardSplit[1][-1].isnumeric()):
                    #overwrite all letters following '_' with
                    cardSplit[1] = "+" + cardSplit[1][-1]
                #if not upgraded, then delete all letters following "_"
                else:
                    cardSplit[1] = ""
            #if card is upgraded
            elif ("+" in card):
                cardSplit = card.split("+")
                if (cardSplit[0] == "Wraith Form v2"):
                    cardSplit[0] = "Wraith Form"
                    cardSplit[1] = "+" + cardSplit[1][-1]
            #card names inconsistent. This card should be
            #Wraith Form
            elif (card == "Wraith Form v2"):
                card = "Wraith Form"
                #append instead of indexing because cardSplit would be empty
                cardSplit.append("Wraith Form")
            
            #if one of the above conditions was true
            if (len(cardSplit) != 0):
                #if card not in uniqueCards
                if not(cardSplit[0] in uniqueCards):
                    deleteIndices4.append(index)
                    #DEBUG: use to print the name of the card that disqualified this row/deck
                    #from being used in the dataset
                    #if (index == 8393):
                    #print(card)
                    break
    index += 1

In [28]:
#combine all indices of rows to be removed
indicesToRemove = set(deleteIndices + deleteIndices1 + deleteIndices2 + deleteIndices3 + deleteIndices4)
len(indicesToRemove)

6246

In [29]:
#remove rows
draftDfv2 = draftDf.drop(labels=indicesToRemove, axis=0, inplace=False)
draftDfv3 = draftDfv2

In [30]:
#Attempted to programmatically edit each string in 'master_deck' but memory
#requirements were too intensive. So, saved the df as csv to manually use the
#excel replace function to format all cards in the 'master_deck' column
#draftDfv3.to_csv('draftDfv3.csv')